In [1]:
import pandas as pd
import json
import requests
import numpy as np

[BC thread](https://basecamp.com/1756858/projects/13683267/messages/67175873)  

Original data tables for adm levels:
* [**adm0**](https://onedrive.live.com/?authkey=%21AB5CsqpH3uRtjwM&cid=5CD64A8A9CA52AC7&id=5CD64A8A9CA52AC7%21330160&parId=5CD64A8A9CA52AC7%21324242&action=locate)
* [**adm1**](https://onedrive.live.com/?authkey=%21AKnPabG3WrQgK08&cid=5CD64A8A9CA52AC7&id=5CD64A8A9CA52AC7%21330161&parId=5CD64A8A9CA52AC7%21324242&action=locate)
* [**adm2**](https://onedrive.live.com/?authkey=%21ALKEsRAkxzpe6sk&cid=5CD64A8A9CA52AC7&id=5CD64A8A9CA52AC7%21330136&parId=5CD64A8A9CA52AC7%21324242&action=locate)
They also provided us coincident tables between [Gaul and Worldbank clases](https://basecamp.com/1756858/projects/13683267/messages/67175873#comment_511809201)


The original geometries where treated with [Mapshaper](http://www.mapshaper.org/) algorithms to reduce their size without compromising their integrity: 
```bash
mapshaper -i indata.shp -simplify visvalingam 10% keep-shapes -o format=shapefile outdata.shp
```

After this first step, the datasets where uploaded to Carto and merged using the below data structure

# New data structure

In [40]:
payload = {'q': "SELECT * FROM gaul_final limit 1"}
r = requests.get('https://simbiotica.carto.com/api/v2/sql', params=payload)
tableStructure= pd.read_json(json.dumps(r.json()['rows']), orient='records')
tableStructure.head(0)


Empty DataFrame
Columns: [adm0_code, adm0_name, adm1_code, adm1_name, adm2_code, adm2_name, cartodb_id, centroid, iso, level, region, the_geom, the_geom_webmercator]
Index: []

The Result table is ```gaul_final``` stored in sim 

# Problematic: missing countries:

We realized that some countries that were present on the first table were missing from the second one.

In [12]:
payload = {'q': "select count(distinct adm0_code) gaul_final_count, level from gaul_final group by level"}
r = requests.get('https://simbiotica.carto.com/api/v2/sql', params=payload)
tableStructure= pd.read_json(json.dumps(r.json()['rows']), orient='records')
tableStructure

gaul_final_count  level
0               181      0
1               278      1
2               279      2

### Original table with the number of all contries and the number of  those that are state Members:

In [6]:
payload = {'q': "SELECT count(cartodb_id) adm0_countries FROM g2015_2014_0_upd270117"}
r = requests.get('https://simbiotica.carto.com/api/v2/sql', params=payload)
tableStructure= pd.read_json(json.dumps(r.json()['rows']), orient='records')
tableStructure.head(1)

adm0_countries
0             278

In [8]:
payload = {'q': "SELECT count(cartodb_id) adm0_countries_M_state FROM g2015_2014_0_upd270117 where status='Member State'"}
r = requests.get('https://simbiotica.carto.com/api/v2/sql', params=payload)
tableStructure= pd.read_json(json.dumps(r.json()['rows']), orient='records')
tableStructure.head(1) 

adm0_countries_m_state
0                     194

### Gaul iso table with the iso per Gaul code

In [4]:
payload = {'q': "SELECT count(cartodb_id) as gaul_iso_countries FROM gaul_iso_table"}
r = requests.get('https://simbiotica.carto.com/api/v2/sql', params=payload)
tableStructure= pd.read_json(json.dumps(r.json()['rows']), orient='records')
tableStructure.head(1)

gaul_iso_countries
0                 196

Discordances between tables

In [49]:
payload = {'q': "with s as (select * from g2015_2014_0_upd270117 where status = 'Member State'), r as (SELECT adm0_name, adm0_code, gaul, iso3, short_name FROM s FULL OUTER JOIN gaul_iso_table on adm0_code=gaul) select * from r where gaul is null or adm0_code is null order by adm0_name asc"}
r = requests.get('https://simbiotica.carto.com/api/v2/sql', params=payload)
tableStructure= pd.read_json(json.dumps(r.json()['rows']), orient='records')
print('count: ', len(tableStructure.index))
tableStructure 

count:  10


adm0_code      adm0_name     gaul  iso3                        short_name
0   147295.0          China      NaN  None                              None
1       86.0  French Guiana      NaN  None                              None
2      146.0  Liechtenstein      NaN  None                              None
3        6.0          Sudan      NaN  None                              None
4        NaN           None    183.0   NIU                              Niue
5        NaN           None  40764.0   SDN                             Sudan
6        NaN           None     60.0   COK                      Cook Islands
7        NaN           None    244.0   TKL        Tokelau (Associate Member)
8        NaN           None     82.0   FRO  Faroe Islands (Associate Member)
9        NaN           None     53.0   CHN                             China

In [64]:
payload = {'q': "with s as (select * from g2015_2014_0_upd270117), r as (SELECT adm0_name, adm0_code, gaul, iso3, short_name FROM s FULL OUTER JOIN gaul_iso_table on adm0_code=gaul) select count(adm0_code) from r where gaul is null or adm0_code is null"}
r = requests.get('https://simbiotica.carto.com/api/v2/sql', params=payload)
tableStructure= pd.read_json(json.dumps(r.json()['rows']), orient='records')
print('count: ', tableStructure['count'][0] )


count:  84


# Possible solutions

1. Keep all countries but we will not have all isos and the gaul codes will produce some conflicts
2. Don't keep them and manually update China and Sudan